# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: ** It is a classification problem since we need to classify two groups, students who need intervention and students who don't need intervention.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [72]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score
from IPython.display import display
%matplotlib inline

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"
display(student_data.head())
student_data.describe()

Student data read successfully!


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,passed
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,no,no,4,3,4,1,1,3,6,no
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,yes,no,5,3,3,1,1,3,4,no
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,yes,no,4,3,2,2,3,3,10,yes
3,GP,F,15,U,GT3,T,4,2,health,services,...,yes,yes,3,2,2,1,1,5,2,yes
4,GP,F,16,U,GT3,T,3,3,other,other,...,no,no,4,3,2,1,2,5,4,yes


,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,Walc,health,absences
count,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000
mean,16.696203,2.749367,2.521519,1.448101,2.035443,0.334177,3.944304,3.235443,3.108861,1.481013,2.291139,3.554430,5.708861
std,1.276043,1.094735,1.088201,0.697505,0.839240,0.743651,0.896659,0.998862,1.113278,0.890741,1.287897,1.390303,8.003096
min,15.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
25%,16.000000,2.000000,2.000000,1.000000,1.000000,0.000000,4.000000,3.000000,2.000000,1.000000,1.000000,3.000000,0.000000
50%,17.000000,3.000000,2.000000,1.000000,2.000000,0.000000,4.000000,3.000000,3.000000,1.000000,2.000000,4.000000,4.000000
75%,18.000000,4.000000,3.000000,2.000000,2.000000,0.000000,5.000000,4.000000,4.000000,2.000000,3.000000,5.000000,8.000000
max,22.000000,4.000000,4.000000,4.000000,4.000000,3.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,75.000000


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [73]:
# TODO: Calculate number of students
n_students = student_data.shape[0]

# TODO: Calculate number of features
n_features = student_data.shape[1] - 1 #remove the target from features

# TODO: Calculate passing students
#n_passed = student_data[ student_data['passed'].isin(['yes'])].shape[0]
n_passed = sum(student_data['passed'] == 'yes')

# TODO: Calculate failing students
#n_failed = student_data[ student_data['passed'] == 'no' ].shape[0]
n_failed = sum(student_data['passed'] == 'no')

# TODO: Calculate graduation rate
#grad_rate = (float(n_passed) / n_students) * 100
grad_rate = 100. * n_passed/n_students

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [74]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [75]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))
display(X_all.head())

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


,school_GP,school_MS,sex_F,sex_M,age,address_R,address_U,famsize_GT3,famsize_LE3,Pstatus_A,...,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences
0,1.0,0.0,1.0,0.0,18,0.0,1.0,1.0,0.0,1.0,...,1,0,0,4,3,4,1,1,3,6
1,1.0,0.0,1.0,0.0,17,0.0,1.0,1.0,0.0,0.0,...,1,1,0,5,3,3,1,1,3,4
2,1.0,0.0,1.0,0.0,15,0.0,1.0,0.0,1.0,0.0,...,1,1,0,4,3,2,2,3,3,10
3,1.0,0.0,1.0,0.0,15,0.0,1.0,1.0,0.0,0.0,...,1,1,1,3,2,2,1,1,5,2
4,1.0,0.0,1.0,0.0,16,0.0,1.0,1.0,0.0,0.0,...,1,0,0,4,3,2,1,2,5,4


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [97]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import train_test_split

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train = None
X_test = None
y_train = None
y_test = None
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=num_test, stratify=y_all, random_state=123)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])
print X_train.shape
print X_test.shape
print "Grad rates:\nTrain set = {:.2f}%".format(100 * (y_train == 'yes').mean())
print "Test set = {:.2f}%".format(100 * (y_test == 'yes').mean())

Training set has 300 samples.
Testing set has 95 samples.
(300, 48)
(95, 48)
Grad rates:
Train set = 67.00%
Test set = 67.37%


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: ** 

*K-Nearest Neighbors*   
KNN is a method used for classification and regression. It is a non-parametric method since it makes no assumption about the relation between variables and result and the probability distribution of the variables. It is conceptually simple and can work with relatively little information to solve complex problems. However, computationally it is expensive; each prediction takes O(dn) where d is the dimension and n is the number of observation.  The number of observations should increase exponentially by increasing the number of features to fill the space. KNN predicts based on the local information, K nearest neighbors, and the events at long distance don’t have any contribution on the prediction. KNN is used in finance and medicine. Combination of KNN with other methods such as SVM makes powerful techniques for prediction. 

*Logistic Regression*    
Logistic regression is used for classification. It is a powerful statistical way of modeling binomial outcome (takes value 0 or 1). The variables can be continuous or discontinuous but I think it works better for continuous variables. The fundamental assumption is that there is a linear relationship between variables and outcome, and therefore it is not a good model for non-linear problems. Logistic regression has many application in industry especially in cases with binomial outcome. For example, it is used to predict whether a patient has a given disease. Logistic regression is a good candidate for this problem since the outcome is binomial

*Support Vector Machines*  
Support vector machines (SVM) is used for classification and regression. The SVM divides the data into two categories by defining a hyper plane which has a maximum margin. SVM can perform linear and non-linear classification using kernel trick. It can be used effectively for high dimensional problem even the number of dimension is greater than the number of sample. It cannot provide probability estimate directly. SVM has many applications such as image classification, text categorization, hand-written characterization, and biology. SVM is a good candidate for this problem since it can classify two groups by a linear or non-linear hyperplane.

*Decision Tree*  
Decision tree is used for classification and regression. It is a non-parametric method and it can handle non-linear problem. It divides the features space into subspaces with similar properties or outcome that are most important to make a decision. It is easy to understand and interpret, and requires little data preparation. It can handle both numerical and categorical data. However, the method is very sensitive to the data; small changes in the data can lead to different splits that undermines the interpretability of the model. Decision trees are not very reproducible on future data. Decision tree has many applications such as astronomy, finance, medicine, and physics. It is a good candidate for this problem since it can handle both numerical and categorical data, and requires little data. 

*Naive Bayes*  
Naïve Bayes is a classification technique based on Bayes’ Theorem with an assumption of independence among predictors. Naive Bayes model is easy to build and particularly useful for very large data sets. It also perform well in multi class prediction. When assumption of independence holds, a Naive Bayes classifier performs better compare to other models like logistic regression and it needs less training data. It perform with categorical and numerical variables. For numerical variable, normal distribution is assumed. It is not good for probability prediction. If categorical variable has a category (in test data set), which was not observed in training data set, then model will assign a 0 (zero) probability and will be unable to make a prediction. Another limitation of Naive Bayes is the assumption of independent predictors. Naive Bayes classifiers mostly used in text classification (due to better result in multi class problems and independence rule) have higher success rate as compared to other algorithms. As a result, it is widely used in Spam filtering (identify spam e-mail) and Sentiment Analysis (in social media analysis, to identify positive and negative customer sentiments). Naive Bayes is not a good candidate for this problem since the features are not independent. However, I would like to test its performance and compare with other methods.


### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [98]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf_model = clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)
    return clf_model
    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [99]:
## KNN
# TODO: Import the three supervised learning models from sklearn
from sklearn.neighbors import KNeighborsClassifier

# TODO: Initialize the three models
clf_A = KNeighborsClassifier(n_neighbors=5)

# TODO: Set up the training set sizes
clf_X_train_100 = train_classifier(clf_A, X_train[0:100], y_train[0:100])
y_train_100 = predict_labels(clf_X_train_100, X_train[0:100], y_train[0:100])
#print y_train_100

clf_X_train_200 = train_classifier(clf_A, X_train[0:200], y_train[0:200])
y_train_200 = predict_labels(clf_X_train_200, X_train[0:200], y_train[0:200])
#print y_train_200

clf_X_train_300 = train_classifier(clf_A, X_train, y_train)
y_train_300 = predict_labels(clf_X_train_300, X_train, y_train)
#print y_train_300

# TODO: Execute the 'train_predict' function for each classifier and each training set size
train_predict(clf_A, X_train[0:100], y_train[0:100], X_test, y_test)
train_predict(clf_A, X_train[0:200], y_train[0:200], X_test, y_test)
train_predict(clf_A, X_train, y_train, X_test, y_test)

Trained model in 0.0010 seconds
Made predictions in 0.0020 seconds.
Trained model in 0.0010 seconds
Made predictions in 0.0040 seconds.
Trained model in 0.0010 seconds
Made predictions in 0.0080 seconds.
Training a KNeighborsClassifier using a training set size of 100. . .
Trained model in 0.0020 seconds
Made predictions in 0.0020 seconds.
F1 score for training set: 0.7538.
Made predictions in 0.0020 seconds.
F1 score for test set: 0.7317.
Training a KNeighborsClassifier using a training set size of 200. . .
Trained model in 0.0020 seconds
Made predictions in 0.0050 seconds.
F1 score for training set: 0.8288.
Made predictions in 0.0030 seconds.
F1 score for test set: 0.8000.
Training a KNeighborsClassifier using a training set size of 300. . .
Trained model in 0.0020 seconds
Made predictions in 0.0100 seconds.
F1 score for training set: 0.8486.
Made predictions in 0.0040 seconds.
F1 score for test set: 0.8028.


### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

The result reported in the tables are based on my first simulation.
** Classifer 1 - KNN**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        0.001            |        0.001           |       0.8060     |       0.7246    |
| 200               |        0.001            |        0.003           |       0.8800     |       0.7692    |
| 300               |        0.001            |        0.001           |       0.8809     |       0.7801    |

** Classifer 2 - Logistic Regression**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |     0.0030              |        0.000           |       0.9104     |     0.7087      |
| 200               |     0.0030              |        0.000           |       0.8421     |     0.7883      |
| 300               |     0.0030              |        0.000           |       0.8435     |     0.7820      |

** Classifer 3 - Decision Trees**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |     0.0010              |        0.001           |       0.8872     |     0.7552      |
| 200               |     0.0010              |        0.001           |       0.8963     |     0.7445      |
| 300               |     0.0010              |        0.001           |       0.8712     |     0.7519      |

** Classifer 3 - SVM**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |     0.0020              |        0.001           |       0.8777     |     0.7746      |
| 200               |     0.0040              |        0.003           |       0.8679     |     0.7815      |
| 300               |     0.0100              |        0.006           |       0.8761     |     0.7838      |

** Classifer 3 - Naive Bayes**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |     0.0030              |        0.000           |       0.8467     |     0.8029      |
| 200               |     0.0010              |        0.001           |       0.8406     |     0.7244      |
| 300               |     0.0010              |        0.000           |       0.8038     |     0.7634      |

In [100]:
# It was suggested by reviewer to simplify the code
# loop thru models, then thru train sizes
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

clf_KNN = KNeighborsClassifier()
clf_LR = LogisticRegression(random_state = 123)
clf_DT = DecisionTreeClassifier(random_state = 123)
clf_SVM = SVC(random_state = 123)
clf_NB = GaussianNB()

for clf in [clf_KNN, clf_LR, clf_DT, clf_SVM, clf_NB]:

    # show the estimator type
    print "\n{}: \n".format(clf.__class__.__name__)

    # loop thru training sizes
    for n in [100, 200, 300]:

        # create training data from first 100, then 200, then 300 points
        train_predict(clf, X_train[:n], y_train[:n], X_test, y_test)


KNeighborsClassifier: 

Training a KNeighborsClassifier using a training set size of 100. . .
Trained model in 0.0000 seconds
Made predictions in 0.0030 seconds.
F1 score for training set: 0.7538.
Made predictions in 0.0020 seconds.
F1 score for test set: 0.7317.
Training a KNeighborsClassifier using a training set size of 200. . .
Trained model in 0.0010 seconds
Made predictions in 0.0030 seconds.
F1 score for training set: 0.8288.
Made predictions in 0.0030 seconds.
F1 score for test set: 0.8000.
Training a KNeighborsClassifier using a training set size of 300. . .
Trained model in 0.0010 seconds
Made predictions in 0.0120 seconds.
F1 score for training set: 0.8486.
Made predictions in 0.0040 seconds.
F1 score for test set: 0.8028.

LogisticRegression: 

Training a LogisticRegression using a training set size of 100. . .
Trained model in 0.0010 seconds
Made predictions in 0.0000 seconds.
F1 score for training set: 0.8548.
Made predictions in 0.0000 seconds.
F1 score for test set: 0.

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Chosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **
The performance of decision trees strongly depends on the training and test data, changing the training and test data significantly changes the F1 test score, indicating that the decision trees is sensitive to the input data and it doesn’t have good performance for small sample. There is no linear correlation between the variables and output since logistic regression and SVM with linear kernel have small F1 scores. KNN has the best performance but I don't thing it is a good method for small number of samples. SVM with rbf kernel has a good performance, and increasing the number of sample improves the performance of the method. In the end, I think SVM is the best model for this problem since it can handle high dimension problem with small number of sample. 

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. For example if you've chosen to use a decision tree or a support vector machine, how does the model go about making a prediction?*

**Answer: **
SVM is a learning algorithm, using existing data to build a model for prediction. In this example, the SVM takes data about previous students (age, gender, family, etc) and their final results, who passed the exam who didn’t. Then it tries to find a gap between students’ data to separate two groups, and defines a boundary in the gap. The gap should be as wide as possible to maximize the distance between two groups. Often, though, it's not easy to draw a decision boundary in low dimensions, so the SVM separates the passing and failing students by line or plane. Using this function created with students we already know passed or not, the SVM can look at new students' data and predict if they will pass or not.  
![title](/notebooks/svm.png "ShowMyImage")
![](https://udacity‐github‐sync‐content.s3.amazonaws.com/_imgs/19273/1468014011/optimal‐hyperplane.png)

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [101]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn import grid_search
from sklearn.metrics import make_scorer

# TODO: Create the parameters list you wish to tune
#parameters = {'kernel' : ['linear', 'poly', 'rbf', 'sigmoid', 'precomputed']}
parameters = {'kernel' : ['linear','rbf', 'poly', 'sigmoid']}
print parameters['kernel']

# TODO: Initialize the classifier
from sklearn.svm import SVC
clf = SVC(random_state = 123)

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label='yes')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = grid_search.GridSearchCV(estimator=clf, param_grid=parameters, scoring=f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_
print clf

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

['linear', 'rbf', 'poly', 'sigmoid']
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='sigmoid',
  max_iter=-1, probability=False, random_state=123, shrinking=True,
  tol=0.001, verbose=False)
Made predictions in 0.0050 seconds.
Tuned model has a training F1 score of 0.8024.
Made predictions in 0.0010 seconds.
Tuned model has a testing F1 score of 0.8050.


In [102]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn import grid_search
from sklearn.metrics import make_scorer

# TODO: Create the parameters list you wish to tune
C_range = np.logspace(-3, 3, 13)
gamma_range = np.logspace(-4, 2, 13)
parameters = dict(gamma=gamma_range, C=C_range)

# TODO: Initialize the classifier
from sklearn.svm import SVC
clf = SVC(random_state = 123)

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label='yes')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = grid_search.GridSearchCV(estimator=clf, param_grid=parameters, scoring=f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_
print clf

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.10000000000000001,
  kernel='rbf', max_iter=-1, probability=False, random_state=123,
  shrinking=True, tol=0.001, verbose=False)
Made predictions in 0.0070 seconds.
Tuned model has a training F1 score of 0.9687.
Made predictions in 0.0020 seconds.
Tuned model has a testing F1 score of 0.8312.


In [103]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn import grid_search
from sklearn.metrics import make_scorer

# Suggestited by reviwer
from sklearn.preprocessing import Normalizer
normer = Normalizer()
X_train = normer.fit_transform(X_train)
X_test = normer.transform(X_test)

# TODO: Create the parameters list you wish to tune
C_range = np.logspace(-3, 3, 13)
gamma_range = np.logspace(-4, 2, 13)
parameters = dict(gamma=gamma_range, C=C_range)

# TODO: Initialize the classifier
from sklearn.svm import SVC
clf = SVC(random_state = 123)

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label='yes')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = grid_search.GridSearchCV(estimator=clf, param_grid=parameters, scoring=f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_
print clf

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

SVC(C=316.2277660168379, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.031622776601683791,
  kernel='rbf', max_iter=-1, probability=False, random_state=123,
  shrinking=True, tol=0.001, verbose=False)
Made predictions in 0.0060 seconds.
Tuned model has a training F1 score of 0.8286.
Made predictions in 0.0020 seconds.
Tuned model has a testing F1 score of 0.8212.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **
Tuning the SVM with rbf kernel model improves the F1 score from 0.8050 to 0.8312. However, normalize the features reduce the F1 score to 0.8212.

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.